###Python setup

1. Install anaconda for Python 2.7

>  http://continuum.io/downloads

2. Create environment

> conda create --name py27 python=2.7 anaconda   

3. Source the environment
   
> source activate py27
   
4. Install pyvmomi

> pip install pyvmomi
    
5. Start python notebook in env

>  ipython notebook

* Object model:
  * https://cdwinvctr01.cdi.com/mob
* Markdown quick ref
  * https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet
  
###Samples
* https://github.com/vmware/pyvmomi-community-samples

###VCenter objects
* `si`
  - A service instance object. The top level object of the vmware world.
    https://github.com/vmware/pyvmomi/blob/master/docs/vim/ServiceInstance.rst 
* `content`
  - service instance content - Access the acutal VM information tree
    https://github.com/vmware/pyvmomi/blob/master/docs/vim/ServiceInstanceContent.rst
* `content.rootFolder`
  - Top level inventory is just a folder object  
    https://github.com/vmware/pyvmomi/blob/master/docs/vim/Folder.rst
* `dc`
  - Datacenter object  
    https://github.com/vmware/pyvmomi/blob/master/docs/vim/Datacenter.rst
* VM
  - https://github.com/vmware/pyvmomi/blob/master/docs/vim/VirtualMachine.rst

In [ ]:
import sys
import ssl
for p in sys.path:
    print p

In [ ]:
from pyVim import connect
from pyVmomi import vmodl
from pyVmomi import vim

In [ ]:
from requests.packages.urllib3 import disable_warnings
disable_warnings()

In [ ]:
uid = r''
pw = r''
host = r''

In [ ]:
default_context = ssl._create_default_https_context
ssl._create_default_https_context = ssl._create_unverified_context
si = connect.SmartConnect(
  host=host,
  user=uid,
  pwd=pw
)
ssl._create_default_https_context = default_context

In [ ]:
def display_snapshot_tree(trees, depth=1):
    for tree in trees:
        # tree: pyVmomi.VmomiSupport.vim.vm.SnapshotTree 
        # tree.snapshot is snapshot object
        ## https://github.com/vmware/pyvmomi/blob/master/docs/vim/vm/Snapshot.rst
        ## tree.snapshot.RevertToSnapshot() revert vm to snapshot
        
        print("  ID:{}".format(tree.id))
        print("    depth  : {}".format(depth))
        print("    Created: {}".format(tree.createTime))
        print("    name   : {}".format(tree.name))
        print("    state  : {}".format(tree.state))

        if tree.childSnapshotList:
            display_snapshot_tree(tree.childSnapshotList, depth+1)

def display_vm(vm):
    if not isinstance(vm, vim.VirtualMachine):
        print("Don't know how to display object")
        print(type(vm))
        return
    
    print("Config")
    print("  hostname: {}".format(vm.guest.hostName))
    print("  UUID: {}".format(vm.config.instanceUuid))
    print("  changeVersion: {}".format(vm.config.changeVersion))
    print("  modified: {}".format(vm.config.modified))
    print("  guestId: {}".format(vm.config.guestId))
    print("Config.Hardware")
    print("  CPUs: {}".format(vm.config.hardware.numCPU))
    print("  memoryMB: {}".format(vm.config.hardware.memoryMB))
    print("Config.Files")
    print("  vmPathName: {}".format(vm.config.files.vmPathName))
    print("  snapshotDirectory: {}".format(vm.config.files.snapshotDirectory))
    print("  logDirectory: {}".format(vm.config.files.logDirectory))
    print("Snapshots")
    if vm.snapshot:
        print("  current: {}".format(vm.snapshot.currentSnapshot.config.changeVersion))
        tree = vm.snapshot.rootSnapshotList
        display_snapshot_tree(tree)
    else:
        print("  None")



##Find VM By host name##
* https://github.com/vmware/pyvmomi/blob/master/docs/vim/SearchIndex.rst

In [ ]:
searchIndex = si.RetrieveServiceContent().searchIndex
type(searchIndex)

In [ ]:
host_name = ''
vm = searchIndex.FindByDnsName(dnsName=host_name, vmSearch=True)

show_methods = False
if show_methods:
    for x in dir(vm):
        print x

# vm: https://github.com/vmware/pyvmomi/blob/master/docs/vim/VirtualMachine.rst
## revertToCurrentSnapshot()

In [ ]:
display_vm(vm)

##Cleanup

In [ ]:
# Cleanup
connect.Disconnect(si)